In [138]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from tqdm.notebook import tqdm

In [139]:
driver = webdriver.Chrome("chromedriver.exe")

In [185]:
df = pd.DataFrame(columns=["아이디", "단지명", "공급/전용", "방수/욕실수", "해당면적 세대수", "현관구조", "세대수", "저/최고층", "준공년월", "세대당주차대수", "용적률", "건폐율", "건설사", "난방", "관리사무소", "주소", "위도", "경도", "배정초", "배정동", "도보"])

In [ ]:
for apt_id in tqdm(range(1, 1000+1)):
# for apt_id in [2]:
    dic = {}

    #url을 불러옵니다.
    driver.get(f"https://new.land.naver.com/complexes/{apt_id}?ms=")
    #"단지정보"를 최대 1초간 기다립니다.
    try:
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//*[@id='summaryInfo']/div[2]/div[2]/button[1]")))
    #"단지정보"가 없으면 다음 apt_id로 넘어갑니다.
    except:
        continue
    #"단지정보"가 있으면 클릭합니다.
    else:
        driver.execute_script("arguments[0].click();", driver.find_element_by_xpath("//*[@id='summaryInfo']/div[2]/div[2]/button[1]"))
        
        for j in [1, 2]:
            #"단지 정보"를 최대 1초간 기다립니다.
            try:
                WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, f"//*[@id='detailContents1']/div[{j}]/div/h5")))
            #"단지 정보"가 없으면 넘어갑니다.
            except:
                continue
                
        for i in [4, 3, 2]:
            #"학군정보"를 클릭합니다.
            try:
                if driver.find_element_by_xpath(f"//*[@id='detailTab{i}']").get_attribute("text") == "학군정보":
                    driver.execute_script("arguments[0].click();", driver.find_element_by_xpath(f"//*[@id='detailTab{i}']"))

                    #데이터를 가져옵니다.
                    time.sleep(1)
                    elem = driver.find_element_by_xpath("//*[@id='detailContents5']/div/div[1]/div[1]/h5").text
                    elem_dong = driver.find_element_by_xpath("//*[@id='detailContents5']/div/div[1]/div[2]/div[1]/div[2]").text
                    elem_time = driver.find_element_by_xpath("//*[@id='detailContents5']/div/div[1]/div[2]/div[2]/div[2]").text
                    dic.update({"배정초":elem, "배정동":elem_dong, "도보":elem_time})
            #"학군정보"가 없으면 넘어갑니다.
            except:
                continue
                
        for k in [1, 2]:
            #"단지정보"를 클릭합니다.
            try:
                if driver.find_element_by_xpath(f"//*[@id='detailTab{k}']").get_attribute("text") == "단지정보":
                    driver.execute_script("arguments[0].click();", driver.find_element_by_xpath(f"//*[@id='detailTab{k}']"))

                    #데이터를 가져옵니다.
                    for r in [1, 2]:
                        try:
                            time.sleep(1)
                            for p, q in zip([1, 1, 2, 2, 3, 3, 4, 5, 6, 7, 7], ["[1]", "[2]", "[1]", "[2]", "[1]", "[2]", "", "", "", "", ""]):
                                col1 = driver.find_element_by_xpath(f"//*[@id='detailContents1']/div[{r}]/table/tbody/tr[{p}]/th{q}").text
                                val1 = driver.find_element_by_xpath(f"//*[@id='detailContents1']/div[{r}]/table/tbody/tr[{p}]/td{q}").text
                                dic.update({col1:val1})
                        except:
                            continue
            #"단지정보"가 없으면 넘어갑니다.
            except:
                continue

        #화살표 버튼이 있으면 클릭합니다
        try:
            driver.find_element_by_xpath("//*[@id='detailContents1']/div[2]/div[2]/div/div[2]/button/span/i").click()
        #화살표 버튼이 없으면 넘어갑니다.
        except:
            pass

        for i in range(0, 20):
            #"단지 내 면적별 정보"의 각 전용면적을 차례대로 클릭합니다.
            try:
                driver.execute_script("arguments[0].click();", driver.find_element_by_xpath(f"//*[@id='tab{i}']/span"))
            #클릭할 전용면적이 없으면 for문을 탈출합니다.
            except:
                break
            #전용면적을 클릭했으면 데이터를 가져옵니다.
            else:
                for l in [1, 2, 3, 4]:
                    col2 = driver.find_element_by_xpath(f"//*[@id='tabpanel']/table/tbody/tr[{l}]/th").text
                    val2 = driver.find_element_by_xpath(f"//*[@id='tabpanel']/table/tbody/tr[{l}]/td").text        
                    dic.update({col2:val2})
                    #"아이디"와 "단지명"을 "dic"에 추가합니다.
                    dic.update({"아이디":apt_id, "단지명":driver.find_element_by_xpath("//*[@id='complexTitle']").text, "위도":re.search(r"(.+)=(\d+.\d+),(\d+.\d+),(.+)", driver.current_url).group(2), "경도":re.search(r"(.+)=(\d+.\d+),(\d+.\d+),(.+)", driver.current_url).group(3)})

                #"df"에 "dic"을 추가하고 중복을 제거합니다.
                df = df.append(dic, ignore_index=True)
                df = df.drop_duplicates()
                
                try:
                    df.to_csv("naver scraper temp.csv", encoding="euc-kr")
                except:
                    continue
print("finished!")

In [184]:
df

,아이디,단지명,공급/전용,방수/욕실수,해당면적 세대수,현관구조,세대수,저/최고층,준공년월,세대당주차대수,...,건폐율,건설사,난방,관리사무소,주소,위도,경도,배정초,배정동,도보
0,2,시영,31.61㎡/28.79㎡(전용률 91%),1개/1개,299세대,복도식,1970세대(총30개동),5층/5층,1984년 06월,0.6대,...,16%,"대우,한라건설","개별난방, 도시가스",02-573-1006,서울시 강남구 개포동 656\n서울시 강남구 언주로 21,37.478961,127.054842,서울구룡초등학교,1~23,도보로 1분
1,2,시영,44.5㎡/40.53㎡(전용률 91%),2개/1개,1000세대,계단식,1970세대(총30개동),5층/5층,1984년 06월,0.6대,...,16%,"대우,한라건설","개별난방, 도시가스",02-573-1006,서울시 강남구 개포동 656\n서울시 강남구 언주로 21,37.478961,127.054842,서울구룡초등학교,1~23,도보로 1분
2,2,시영,55.76㎡/50.78㎡(전용률 91%),2개/1개,180세대,계단식,1970세대(총30개동),5층/5층,1984년 06월,0.6대,...,16%,"대우,한라건설","개별난방, 도시가스",02-573-1006,서울시 강남구 개포동 656\n서울시 강남구 언주로 21,37.478961,127.054842,서울구룡초등학교,1~23,도보로 1분
3,2,시영,56.2㎡/51.18㎡(전용률 91%),2개/1개,300세대,계단식,1970세대(총30개동),5층/5층,1984년 06월,0.6대,...,16%,"대우,한라건설","개별난방, 도시가스",02-573-1006,서울시 강남구 개포동 656\n서울시 강남구 언주로 21,37.478961,127.054842,서울구룡초등학교,1~23,도보로 1분
4,2,시영,63.05㎡/57.42㎡(전용률 91%),3개/1개,191세대,계단식,1970세대(총30개동),5층/5층,1984년 06월,0.6대,...,16%,"대우,한라건설","개별난방, 도시가스",02-573-1006,서울시 강남구 개포동 656\n서울시 강남구 언주로 21,37.478961,127.054842,서울구룡초등학교,1~23,도보로 1분
